# Bayesian Model Averaging (BMA) with Expectation-Maximization (EM) Algorithm

The Bayesian Model Averaging (BMA) method is applied to combine forecasts from multiple models to produce a more robust and accurate forecast. The Expectation-Maximization (EM) algorithm is used to iteratively optimize the weights and variances of each model.

## Function Overview

```{python} 
def bma_em(y, forecasted_y, max_iterations=1000, tol=1e-8):
```

**Parameters**:
- `y`: Observed values
- `forecasted_y`: Forecasts from different models as a 2D array, where each column represents a different model.
- `max_iterations`: Maximum number of iterations for the EM algorithm.
- `tol`: Convergence tolerance to stop the algorithm once improvements become negligible.

## Initialization

 ```{python} 
T, K = forecasted_y.shape
w = np.full(K, 1/K)
sigma = np.array([np.std(y - forecasted_y[:, k]) for k in range(K)])
```

- `T` is the number of observations, and `K` is the number of models.
- `w` contains the initial weights for each model, uniformly distributed.
- `sigma` contains the initial standard deviations for each model, computed as the standard deviation of the difference between the observed values and each model's forecast.

## EM Algorithm Loop

1. **E-step**: Computes the expectation of the latent variables based on current parameters.

    For each observation and model, we calculate the responsibility (or weight) of that model for the current observation. The responsibility is proportional to the likelihood of the observation under the model's current parameters. These responsibilities are stored in the `z` matrix.

2. **M-step**: Updates the parameters based on the responsibilities computed in the E-step.

    Weights for each model are updated by taking the average of their responsibilities over all observations.
    The variance (`sigma`) for each model is updated by considering the weighted variance of the model's forecast errors.

3. **Log-likelihood computation**: The log-likelihood measures the likelihood of the observed data given the current parameters. It's used as a convergence criterion. The algorithm stops when the increase in log-likelihood is smaller than the tolerance `tol`.

## Outputs

The function returns the optimized weights `w` and variances `sigma` for each model.

---

## Testing with Random Data

After the function, random datasets are generated for testing. The dataset includes 100 observations (`y`) and forecasts from 3 models (`forecasted_y`). The function is then executed with this data, and the optimized weights and sigma values are printed.

---

## Validation and Visualization

### Root Mean Square Error (RMSE)
The RMSE gives a sense of the magnitude of the error between the forecasted values and observed values. It's commonly used because it punishes large errors more severely than smaller ones.

### Skill Score
The skill score is a measure of the relative improvement of one forecast over another. It can be used to compare the performance of the BMA forecast to individual model forecasts or a simple average of the model forecasts.

### Continuous Rank Probability Score (CRPS)
CRPS generalizes the RMSE to probabilistic forecasts. It provides a measure of the difference between the forecasted probabilities and the observed outcomes.

### Attributes Diagram
The attributes diagram is a graphical representation that allows one to assess the reliability, resolution, and sharpness of probabilistic forecasts.

### Leave-Two-Out Cross-Validation
Given the autocorrelation in the forecasts, a leave-two-out cross-validation is applied. This means that for each pair of observations left out, the model is trained on the remaining data and its performance is evaluated on the left-out pair. This approach provides a robust estimate of the model's performance.

---

In [ ]:
import numpy as np

def bma_em(y, forecasted_y, max_iterations=1000, tol=1e-8):
    """
    Bayesian Model Averaging using EM Algorithm.
    
    Parameters:
    - y: observed values
    - forecasted_y: forecasted values from different models (2D array)
    - max_iterations: Maximum number of iterations for the EM algorithm
    - tol: Convergence tolerance
    
    Returns:
    - w: Weights for each model
    - sigma: Variance for each model's forecast
    """
    T, K = forecasted_y.shape
    w = np.full(K, 1/K)  # Initialize weights uniformly
    sigma = np.array([np.std(y - forecasted_y[:, k]) for k in range(K)])  # Initialize variances
    
    prev_L = -np.inf
    
    for iteration in range(max_iterations):
        # E-step
        z = np.zeros((T, K))
        for t in range(T):
            total = sum([w[k] * (1 / (sigma[k] * np.sqrt(2 * np.pi))) * 
                         np.exp(-0.5 * ((y[t] - forecasted_y[t, k]) / sigma[k])**2) for k in range(K)])
            for k in range(K):
                z[t, k] = (w[k] * (1 / (sigma[k] * np.sqrt(2 * np.pi))) * 
                          np.exp(-0.5 * ((y[t] - forecasted_y[t, k]) / sigma[k])**2)) / total
        
        # M-step
        w = z.mean(axis=0)
        for k in range(K):
            sigma[k] = np.sqrt(sum(z[:, k] * (y - forecasted_y[:, k])**2) / sum(z[:, k]))
        
        # Log-likelihood
        L = sum([np.log(sum([w[k] * (1 / (sigma[k] * np.sqrt(2 * np.pi))) * 
                            np.exp(-0.5 * ((y[t] - forecasted_y[t, k]) / sigma[k])**2) for k in range(K)])) for t in range(T)])
        
        # Convergence check
        if L - prev_L < tol:
            break
        prev_L = L
    
    return w, sigma

### Example Usage

In [6]:
# Generate random data
np.random.seed(0)  # For reproducibility
y = np.random.randn(100)
forecasted_y = np.random.randn(100, 3)  # Sample forecasts from 3 models

w, sigma = bma_em(y, forecasted_y)
print("Weights:", w)
print("Sigma:", sigma)


Weights: [0.39385837 0.14429187 0.46184976]
Sigma: [1.24849487 0.33603718 0.91692496]


### Validation

In [7]:
from sklearn.metrics import mean_squared_error
from scipy.integrate import quad

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def skill_score(y_true, y_pred):
    mse_pred = mean_squared_error(y_true, y_pred)
    climatology = np.mean(y_true)
    mse_clim = mean_squared_error(y_true, [climatology]*len(y_true))
    ss = 1 - mse_pred / mse_clim   
    return ss


def crps(y_true, y_pred, sigma):
    norm = (y_true - y_pred) / sigma
    first_term = y_true - y_pred
    second_term = sigma * (2 * norm.cdf(norm) - 1)
    third_term = 2 * norm.pdf(norm)
    fourth_term = (norm ** 2)
    return first_term * (second_term - third_term + fourth_term)

def leave_two_out_cv(y, forecasted_y, func):
    n = len(y)
    results = []

    for i in range(0, n, 2):
        y_train = np.delete(y, [i, i+1])
        forecasted_y_train = np.delete(forecasted_y, [i, i+1], axis=0)

        y_test = y[i:i+2]
        forecasted_y_test = forecasted_y[i:i+2]

        w, sigma = func(y_train, forecasted_y_train)
        y_pred = np.dot(w, forecasted_y_test.T)
        
        results.append((y_test, y_pred))
    return results

cv_results = leave_two_out_cv(y, forecasted_y, bma_em)
rmses = [rmse(y_test, y_pred) for y_test, y_pred in cv_results]
print("RMSE:", np.mean(rmses))


RMSE: 1.08963898479372
